In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

# data preprocessing 
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV

# ML models
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingClassifier
from sklearn.cluster import KMeans

# model scoring
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, confusion_matrix, classification_report, silhouette_score

# hyperparametes
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_val_score

# from imblearn.over_sampling import SMOTE

# Personalized Nutrition System: 

## Issued by: Vesela Vekieva

## Introduction

## Зареждане на Data Sets

In [3]:
directory_path = '../data/'

# Зареждане на първия датасет (dietry_habits_cleaned) от pickle
with open(directory_path + 'dietry_habits_cleaned.pkl', 'rb') as f:
    dietry_habits_cleaned = pickle.load(f)

# Зареждане на втория датасет (food_macronutrients_cleaned) от pickle
with open(directory_path + 'food_macronutrients_cleaned.pkl', 'rb') as f:
    food_macronutrients_cleaned = pickle.load(f)

In [4]:
dietry_habits_cleaned

,gender,age,height_cm,weight_kg,bmi,diet_type,daily_fruit_intake,daily_vegetable_intake,daily_meal_count,snack_frequency,...,soft_drinks_intake_l,daily_coffee_consumption,alcohol_consumption,food_allergies,food_intolerances,smoking_status,weekly_exercise_frequency,health_condition,sleep_quality,diet_type_encoded
0,Female,69,159,61.4,24.3,Vegetarian,2,4,1,Never,...,0.76,0,Moderate,No,No,Former Smoker,5,Hypertension,Fair,4
1,Male,25,172,79.6,26.9,Flexitarian,0,0,4,Sometimes,...,0.24,1,Never,No,No,Current Smoker,1,Healthy,Very Good,0
2,Male,46,180,86.5,26.7,Omnivorous,1,2,5,Sometimes,...,0.18,1,Occasional,No,No,Current Smoker,3,Heart Disease,Good,1
3,Female,49,163,70.4,26.5,Flexitarian,0,3,1,Rarely,...,0.21,0,Moderate,No,No,Current Smoker,1,Healthy,Poor,0
4,Female,45,178,71.9,22.7,Omnivorous,0,1,4,Never,...,0.47,3,Moderate,No,No,Former Smoker,1,Healthy,Fair,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5070,Female,34,168,66.0,23.4,Pescatarian,0,3,3,Always,...,0.80,0,Occasional,No,No,Non-Smoker,1,Healthy,Fair,2
5076,Female,67,169,83.6,27.3,Omnivorous,2,0,5,Rarely,...,0.29,3,Moderate,No,No,Former Smoker,1,Diabetes,Fair,1
5079,Female,22,175,65.2,21.3,Omnivorous,4,0,2,Sometimes,...,0.49,1,Moderate,No,Lactose,Non-Smoker,5,Diabetes,Poor,1
5086,Male,54,168,60.1,21.3,Omnivorous,0,3,3,Always,...,0.98,1,Moderate,No,No,Former Smoker,3,Healthy,Fair,1


In [5]:
food_macronutrients_cleaned

,name,type,energy_kcal,water_g,protein_g,total_fat_g,carbohydrates_g,fiber_g,sugars_g,description
0,Apple,fruit,48,86.70,0.27,0.13,12.70,1.3,10.10,facts
1,Apricot,fruit,48,86.40,1.40,0.39,11.12,2.0,9.24,facts
2,Avocado,fruit,160,73.23,2.00,14.70,8.53,6.7,0.66,
3,Banana,fruit,89,74.91,1.09,0.33,22.84,2.6,12.23,facts
4,Blackberries,fruit,43,88.15,1.39,0.49,9.61,5.3,4.88,
...,...,...,...,...,...,...,...,...,...,...
3409,Puddings,added sugars,365,6.80,10.08,3.00,74.42,10.1,0.70,"choc flavor,lo cal,reg,dry mix"
3410,Puddings,added sugars,351,10.40,1.60,0.10,86.04,0.9,2.90,"all flavors xcpt choc,lo cal,reg,dry mix"
3411,Puddings,added sugars,350,6.84,0.81,0.90,84.66,0.8,0.90,"all flavors xcpt choc,lo cal,inst,dry mix"
3412,Fruit-flavored drk mix,beverages,226,0.96,0.00,0.00,91.30,0.0,0.00,"pdr,unswtnd"


## Data Preprocessing and Feature Engineering

In [6]:
dietry_habits_normalized = dietry_habits_cleaned.copy()

In [7]:
dietry_habits_normalized.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5010 entries, 0 to 5087
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   gender                     5010 non-null   object 
 1   age                        5010 non-null   int64  
 2   height_cm                  5010 non-null   int64  
 3   weight_kg                  5010 non-null   float64
 4   bmi                        5010 non-null   float64
 5   diet_type                  5010 non-null   object 
 6   daily_fruit_intake         5010 non-null   int64  
 7   daily_vegetable_intake     5010 non-null   int64  
 8   daily_meal_count           5010 non-null   int64  
 9   snack_frequency            5010 non-null   object 
 10  water_intake_l             5010 non-null   float64
 11  soft_drinks_intake_l       5010 non-null   float64
 12  daily_coffee_consumption   5010 non-null   int64  
 13  alcohol_consumption        5010 non-null   object 
 1

In [8]:
dietry_habits_normalized.columns

Index(['gender', 'age', 'height_cm', 'weight_kg', 'bmi', 'diet_type',
       'daily_fruit_intake', 'daily_vegetable_intake', 'daily_meal_count',
       'snack_frequency', 'water_intake_l', 'soft_drinks_intake_l',
       'daily_coffee_consumption', 'alcohol_consumption', 'food_allergies',
       'food_intolerances', 'smoking_status', 'weekly_exercise_frequency',
       'health_condition', 'sleep_quality', 'diet_type_encoded'],
      dtype='object')

In [9]:
dietry_habits_normalized.duplicated().sum()

0

In [10]:
dietry_habits_normalized.isnull().sum()

gender                       0
age                          0
height_cm                    0
weight_kg                    0
bmi                          0
diet_type                    0
daily_fruit_intake           0
daily_vegetable_intake       0
daily_meal_count             0
snack_frequency              0
water_intake_l               0
soft_drinks_intake_l         0
daily_coffee_consumption     0
alcohol_consumption          0
food_allergies               0
food_intolerances            0
smoking_status               0
weekly_exercise_frequency    0
health_condition             0
sleep_quality                0
diet_type_encoded            0
dtype: int64

In [11]:
food_macronutrients_normalized = food_macronutrients_cleaned.copy()

In [12]:
food_macronutrients_normalized.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3414 entries, 0 to 3413
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             3414 non-null   object 
 1   type             3414 non-null   object 
 2   energy_kcal      3414 non-null   int64  
 3   water_g          3414 non-null   float64
 4   protein_g        3414 non-null   float64
 5   total_fat_g      3414 non-null   float64
 6   carbohydrates_g  3414 non-null   float64
 7   fiber_g          3414 non-null   float64
 8   sugars_g         3414 non-null   float64
 9   description      3414 non-null   object 
dtypes: float64(6), int64(1), object(3)
memory usage: 266.8+ KB


In [13]:
food_macronutrients_normalized.columns

Index(['name', 'type', 'energy_kcal', 'water_g', 'protein_g', 'total_fat_g',
       'carbohydrates_g', 'fiber_g', 'sugars_g', 'description'],
      dtype='object')

In [14]:
food_macronutrients_normalized.duplicated().sum()

0

In [15]:
food_macronutrients_normalized.isnull().sum()

name               0
type               0
energy_kcal        0
water_g            0
protein_g          0
total_fat_g        0
carbohydrates_g    0
fiber_g            0
sugars_g           0
description        0
dtype: int64

### Кодиране на категорийни променливи и нормализация на числови данни

1. Кодиране на категорийни променливи

Причина: Алгоритмите като Decision Trees и Random Forest могат да работят директно с категорийни данни, но обикновено се препоръчва да ги трансформираме в числови стойности, за да улесним процеса на моделиране и да избегнем потенциални грешки.

Обяснение:
Категорийните колони, като 'gender', 'diet_type', 'snack_frequency', 'smoking_status', alcohol_consumption', 'food_allergies', и др., трябва да бъдат преобразувани в числов формат. За целта можем да използваме LabelEncoder или pd.get_dummies().

2. Нормализация на числови данни

Причина: Нормализацията помага на моделите да се представят по-добре и да конвергират по-бързо, особено при алгоритми, чувствителни към мащабите на данните.
Обяснение:
За числовите колони като 'age', 'height_cm', 'weight_kg', 'bmi', 'water_intake_l', 'soft_drinks_intake_l' можем да използваме Min-Max Scaling или Standard Scaling.


#### dietry_habits_cleaned

In [16]:
dietry_habits_normalized.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5010 entries, 0 to 5087
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   gender                     5010 non-null   object 
 1   age                        5010 non-null   int64  
 2   height_cm                  5010 non-null   int64  
 3   weight_kg                  5010 non-null   float64
 4   bmi                        5010 non-null   float64
 5   diet_type                  5010 non-null   object 
 6   daily_fruit_intake         5010 non-null   int64  
 7   daily_vegetable_intake     5010 non-null   int64  
 8   daily_meal_count           5010 non-null   int64  
 9   snack_frequency            5010 non-null   object 
 10  water_intake_l             5010 non-null   float64
 11  soft_drinks_intake_l       5010 non-null   float64
 12  daily_coffee_consumption   5010 non-null   int64  
 13  alcohol_consumption        5010 non-null   object 
 1

In [17]:
# Create an instance of LabelEncoder
label_encoder = LabelEncoder()

# Columns for encoding
categorical_columns = ['gender', 'diet_type', 'snack_frequency', 'alcohol_consumption', 
                       'food_allergies', 'food_intolerances', 'smoking_status', 'health_condition', 'sleep_quality']

# Apply LabelEncoder to each column of the list
for col in categorical_columns:
    dietry_habits_normalized[col + '_encoded'] = label_encoder.fit_transform(dietry_habits_normalized[col])

# След като добавиш кодираните колони, изтрий оригиналните категориални колони
dietry_habits_normalized = dietry_habits_normalized.drop(categorical_columns, axis=1)

In [18]:
# Create an instance of MinMaxScaler
scaler = MinMaxScaler()

# Select numerical columns for scaling
columns_to_normalize = ['age', 'height_cm', 'weight_kg', 'bmi', 'water_intake_l', 'soft_drinks_intake_l']

# Scale the numerical columns
dietry_habits_normalized[columns_to_normalize] = scaler.fit_transform(dietry_habits_normalized[columns_to_normalize])

In [19]:
dietry_habits_normalized

,age,height_cm,weight_kg,bmi,daily_fruit_intake,daily_vegetable_intake,daily_meal_count,water_intake_l,soft_drinks_intake_l,daily_coffee_consumption,weekly_exercise_frequency,diet_type_encoded,gender_encoded,snack_frequency_encoded,alcohol_consumption_encoded,food_allergies_encoded,food_intolerances_encoded,smoking_status_encoded,health_condition_encoded,sleep_quality_encoded
0,1.000000,0.225,0.285181,0.45625,2,4,1,0.076,0.76,0,5,4,0,1,1,3,3,1,3,1
1,0.137255,0.550,0.511831,0.61875,0,0,4,0.840,0.24,1,1,0,1,4,2,3,3,0,1,4
2,0.549020,0.750,0.597758,0.60625,1,2,5,0.252,0.18,1,3,1,1,4,3,3,3,0,2,2
3,0.607843,0.325,0.397260,0.59375,0,3,1,0.188,0.21,0,1,0,0,3,1,3,3,0,1,3
4,0.529412,0.700,0.415940,0.35625,0,1,4,0.164,0.47,3,1,1,0,1,1,3,3,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5070,0.313725,0.450,0.342466,0.40000,0,3,3,0.852,0.80,0,1,2,0,0,3,3,3,2,1,1
5076,0.960784,0.475,0.561644,0.64375,2,0,5,0.092,0.29,3,1,1,0,3,1,3,3,1,0,1
5079,0.078431,0.625,0.332503,0.26875,4,0,2,0.152,0.49,1,5,1,0,4,1,3,2,2,0,3
5086,0.705882,0.450,0.268991,0.26875,0,3,3,0.276,0.98,1,3,1,1,0,1,3,3,1,1,1


#### food_macronutrients_cleaned

In [20]:
food_macronutrients_normalized.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3414 entries, 0 to 3413
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             3414 non-null   object 
 1   type             3414 non-null   object 
 2   energy_kcal      3414 non-null   int64  
 3   water_g          3414 non-null   float64
 4   protein_g        3414 non-null   float64
 5   total_fat_g      3414 non-null   float64
 6   carbohydrates_g  3414 non-null   float64
 7   fiber_g          3414 non-null   float64
 8   sugars_g         3414 non-null   float64
 9   description      3414 non-null   object 
dtypes: float64(6), int64(1), object(3)
memory usage: 266.8+ KB


In [21]:
# Columns for encoding
label_encoder = LabelEncoder()
food_macronutrients_normalized['name_encoded'] = label_encoder.fit_transform(food_macronutrients_normalized['name'])
food_macronutrients_normalized['type_encoded'] = label_encoder.fit_transform(food_macronutrients_normalized['type'])

# Scale the numerical columns
scaler = MinMaxScaler()
columns_to_normalize = ['energy_kcal', 'water_g', 'protein_g', 'total_fat_g', 'carbohydrates_g', 'fiber_g', 'sugars_g']
food_macronutrients_normalized[columns_to_normalize] = scaler.fit_transform(food_macronutrients_normalized[columns_to_normalize])

In [22]:
food_macronutrients_normalized

,name,type,energy_kcal,water_g,protein_g,total_fat_g,carbohydrates_g,fiber_g,sugars_g,description,name_encoded,type_encoded
0,Apple,fruit,0.053215,0.8670,0.003277,0.0013,0.128153,0.016456,0.105815,facts,7,5
1,Apricot,fruit,0.053215,0.8640,0.016990,0.0039,0.112210,0.025316,0.096805,facts,10,5
2,Avocado,fruit,0.177384,0.7323,0.024272,0.1470,0.086075,0.084810,0.006915,,15,5
3,Banana,fruit,0.098670,0.7491,0.013228,0.0033,0.230474,0.032911,0.128130,facts,18,5
4,Blackberries,fruit,0.047672,0.8815,0.016869,0.0049,0.096973,0.067089,0.051126,,31,5
...,...,...,...,...,...,...,...,...,...,...,...,...
3409,Puddings,added sugars,0.404656,0.0680,0.122330,0.0300,0.750959,0.127848,0.007334,"choc flavor,lo cal,reg,dry mix",326,0
3410,Puddings,added sugars,0.389135,0.1040,0.019417,0.0010,0.868214,0.011392,0.030382,"all flavors xcpt choc,lo cal,reg,dry mix",326,0
3411,Puddings,added sugars,0.388027,0.0684,0.009830,0.0090,0.854289,0.010127,0.009429,"all flavors xcpt choc,lo cal,inst,dry mix",326,0
3412,Fruit-flavored drk mix,beverages,0.250554,0.0096,0.000000,0.0000,0.921292,0.000000,0.000000,"pdr,unswtnd",174,1


## Обединяване на данните

Тъй като диетата ще бъде основната ключова променлива, ще трябва да създадем логика, която да свързва всяка диета с конкретни храни, които са подходящи за нея. Това включва не само вида на храната, но и макронутриентите, хранителните алергии, непоносимостите и здравословното състояние на потребителя.

Обединяване на основата на diet_type: Ще съединим двата датасета, като използваме diet_type като ключова променлива. Това ще позволи да съберем информацията за хранителните стойности с данните за диетичните навици.

Филтриране на данните по алергии и нетолерантности: След като сме обединили данните, ще филтрираме храните, които съдържат алергени или компоненти, които потребителят не може да консумира (според колоните food_allergies и food_intolerances).

Филтриране по здравословно състояние: Ще приложим допълнителен филтър за здравословното състояние на потребителя, за да изключим храни, които не са подходящи в зависимост от здравословния му статус.

Този код ще филтрира храните в зависимост от диетата, която е избрана от потребителя. Тъй като можем да имаме множество категории храни, е важно да подготвим съответната мапинг таблица (като примерната по-горе) за всяка диета.

Филтриране по алергии:

Премахваме храни, които съдържат алергени, изброени в списъка на потребителя.
Филтриране по непоносимости:

Изключваме храни, които съдържат съставки, към които потребителят има непоносимост.
Филтриране по здравословно състояние:

За всяко здравословно състояние прилагаме специфични филтри. Например:
За диабет се ограничава съдържанието на захари и въглехидрати, като се предпочитат храни с високо съдържание на протеини и фибри.
За високо кръвно налягане се избягват храни с високо съдържание на захар и се предпочитат храни с повече фибри.
Крайни препоръки:

Резултатът е списък с храни, които преминават през всички филтри и са подходящи за конкретния потребител.

1. Hypertension (Хипертония)
За хора с хипертония е важно да се наблюдават калориите, захарите и фибрите в храната, тъй като високото калорийно съдържание може да доведе до покачване на кръвното налягане, а високото съдържание на захар може да влоши състоянието.

Енергия (калории): Трябва да се избягват храни с високо калорично съдържание. За целта можем да поставим праг под 500 kcal.

$$\text{energy\_kcal} < 500 \Rightarrow \frac{500-0}{902-0} \approx 0.554$$

Захари: Трябва да се избягват храни с високо съдържание на захар. Например, прага може да бъде под 10 г.

$$\text{sugars\_g} < 10 \Rightarrow \frac{95.45 - 0}{10 - 0} \approx 0.105$$


2. Heart Disease (Сърдечни заболявания)
При сърдечни заболявания е важно да се избягват храни с високо съдържание на мазнини и захар, както и да се приемат повече фибри и протеини.

Общо мазнини: За сърдечни заболявания трябва да се избягват храни с високо съдържание на мазнини. Можем да поставим праг за общите мазнини под 15 г.

$$\text{total\_fat\_g} < 15 \Rightarrow \frac{100 - 0}{15 - 0} \approx 0.15$$

Захари: Подобно на хипертонията, трябва да се избягват храни с високо съдържание на захар. Прагът може да бъде под 5 г.
$$\text{sugars\_g} < 5 \Rightarrow \frac{95.45 - 0}{5 - 0} \approx 0.052$$


3. Diabetes (Диабет)
При диабет е важно да се ограничи количеството на захарите и въглехидратите, тъй като те водят до покачване на нивата на кръвната захар. Препоръчително е също така храната да съдържа достатъчно фибри и протеини, които могат да помогнат за контролирането на глюкозата в кръвта.

Захари: Трябва да се избягват храни с високо съдържание на захар. Например, прагът може да бъде под 5 г.
$$\text{sugars\_g} < 5 \Rightarrow \frac{95.45 - 0}{5 - 0} \approx 0.052$$

Въглехидрати: Потребителите с диабет трябва да избягват храни с високо съдържание на въглехидрати. Прагът може да бъде под 30 г.

$$\text{carbohydrates\_g} < 30 \Rightarrow \frac{99.1 - 0}{30 - 0} \approx 0.303$$

In [23]:
food_macronutrients_normalized['type'].value_counts()

type
meat                   1479
grains                  656
added sugars            312
beverages               285
dairy                   211
oils and solid fats     162
vegetable                94
fruit                    52
fish                     49
shellfish                44
egg                      42
plant-based drink        14
vegetarian               14
Name: count, dtype: int64

In [24]:
# Групиране по категория 'type' и филтриране само за 'added sugars'
added_sugars_data = food_macronutrients_normalized[food_macronutrients_normalized['type'] == 'fruit']

# Получаване на уникалните имена за категорията 'added sugars'
unique_added_sugars = added_sugars_data['name'].unique()

# Печат на резултата
print(unique_added_sugars)

['Apple' 'Apricot' 'Avocado' 'Banana' 'Blackberries' 'Blueberry'
 'Carambola' 'Cherimoya fruit' 'Cherry fruit' 'Clementine' 'Cranberries'
 'Currants Black' 'Currants' 'Date nutrition: Dates' 'Dates' 'Durian'
 'Fig nutrition: Figs' 'Grapefruit' 'Grape nutrition: Grapes'
 'Groundcherries' 'Guava' 'Gooseberries' 'Jackfruit'
 'Kiwi nutriion: Kiwifruit' 'Kumquat' 'Lemon nutrition' 'Lime'
 'Litchis or Lychees' 'Mandarin' 'Mango' 'Melons' 'Mulberries' 'Nectarine'
 'Oranges' 'Papaya' 'Passion fruit' 'Peaches' 'Pear'
 'Persimmon nutrition: Persimmons' 'Pineapple' 'Plum' 'Pomegranate'
 'Quince fruit' 'Raspberry' 'Strawberry' 'Tangerine' 'Watermelon']


In [25]:
# 1. Създаване на речник за типове диети
diet_food_mapping = {
    'Vegetarian': ['fruit', 'vegetable', 'oils and solid fats', 'dairy', 'egg', 'added sugars', 'grains', 'beverages', 'plant-based drink', 'vegetarian'],
    'Flexitarian': ['fruit', 'vegetable', 'oils and solid fats', 'dairy', 'egg', 'added sugars', 'meat', 'grains', 'beverages', 'shellfish', 'plant-based drink', 'fish', 'vegetarian'],
    'Omnivorous': ['fruit', 'vegetable', 'oils and solid fats', 'dairy', 'egg', 'added sugars', 'meat', 'grains', 'beverages', 'shellfish', 'plant-based drink', 'fish', 'vegetarian'],
    'Pescatarian': ['fruit', 'vegetable', 'oils and solid fats', 'dairy', 'egg', 'added sugars', 'grains', 'beverages', 'shellfish', 'plant-based drink', 'fish', 'vegetarian'],
    'Vegan': ['fruit', 'vegetable', 'grains', 'beverages', 'plant-based drink']
}

# 2. Примерни входни данни от потребителя
user_diet_type = 'Omnivorous'  # Примерен тип диета
user_allergies = ['No']  # Примерни алергии
user_intolerances = ['No']  # Примерни непоносимости
user_health_condition = 'No'  # Примерно здравословно състояние

# Списък с храни, които съдържат алергени
food_allergen_map = {
    'Peanuts': ['Peanut Butter', 'Peanuts'],
    'Shellfish': ['Crab' 'Lobster' 'Shrimp' 'Spiny lobster', 'Clam', 'Octopus', 'Oyster', 'Scallop', 'Squid'],
    'Dairy': ['Cheese', 'Cheese fd', 'Cheese sprd', 'Cream', 'Milk', 'Milk shakes', 'Yogurt', 'Cheese sub',
              'Cheese sau', 'Sour cream', 'Ice creams', 'Frozen yogurts', 'Kraft macaroni & cheese dinner original flavor', 
              'Macaroni and cheese', 'Creamy drsng', 'Milk dssrt', 'Cheese product', 'Cheese food'],
    'Eggs': ['Egg', 'Egg mix', 'Eggnog-flavor mix', 'Egg custards', 'Scrambled eggs&sausage w/hashed brown potatoes', 'Egg rolls', 'Eggs'],
    'Soybeans': ['Tofu', 'Soymilk', 'Soymilk (all flavors)', 'Soy sau made from soy&wheat (shoyu)'],
    'Sesame': ['Tahini', 'Sesame seeds', 'Sesame'],
    'Wheat': ['Cereals rte', 'Cereals', 'Cereals ready to eat', 'Cereal wafer straws', 'Bagel', 'Bagels', 'Tortillas', 'Tortilla chips',
             'Wheat bran', 'Wheat germ', 'Wheat flour', 'Wheat flr', 'Macaroni', 'Wheat flours'],
    'Fish': ['Salmon', 'Sea bass', 'Shark', 'Tuna', 'Whitefish', 'Salmonberries'],
    # Добавете други алергени според нуждите
}

# Списък с храни, които съдържат непоносимости
food_intolerance_map = {
    'Lactose': ['Cheese', 'Cheese fd', 'Cheese sprd', 'Cream', 'Milk', 'Milk shakes', 'Yogurt', 'Cheese sub',
              'Cheese sau', 'Sour cream', 'Ice creams', 'Frozen yogurts', 'Kraft macaroni & cheese dinner original flavor', 
              'Macaroni and cheese', 'Creamy drsng', 'Milk dssrt', 'Cheese product', 'Cheese food'],
    'Gluten': ['Cereals rte', 'Cereals', 'Cereals ready to eat', 'Cereal wafer straws', 'Bagel', 'Bagels', 'Tortillas', 'Tortilla chips',
             'Wheat bran', 'Wheat germ', 'Wheat flour', 'Wheat flr', 'Macaroni', 'Wheat flours'],
    'Fructose': ['Apple', 'Apricot', 'Avocado', 'Banana', 'Blackberries', 'Blueberry', 'Carambola', 'Cherimoya fruit', 'Cherry fruit', 
                 'Clementine', 'Cranberries', 'Currants Black', 'Currants', 'Date nutrition: Dates', 'Dates', 'Durian', 'Fig nutrition: Figs',
                 'Grapefruit', 'Grape nutrition: Grapes', 'Groundcherries', 'Guava', 'Gooseberries', 'Jackfruit', 'Kiwi nutriion: Kiwifruit',
                 'Kumquat', 'Lemon nutrition', 'Lime', 'Litchis or Lychees', 'Mandarin', 'Mango', 'Melons', 'Mulberries', 'Nectarine', 'Oranges',
                 'Papaya', 'Passion fruit', 'Peaches', 'Pear', 'Persimmon nutrition: Persimmons', 'Pineapple', 'Plum', 'Pomegranate', 'Quince fruit', 
                 'Raspberry', 'Strawberry', 'Tangerine', 'Watermelon'],
}

# 3. Филтриране на храните според тип диета
# Извеждаме подходящите храни според диетата на потребителя
suitable_foods = food_macronutrients_normalized[food_macronutrients_normalized['type'].isin(diet_food_mapping[user_diet_type])]

# 4. Филтриране по алергии
if user_allergies != ['No']:  # Проверка дали има алергии
    filtered_by_allergies = suitable_foods[~suitable_foods['name'].isin([item for allergen in user_allergies for item in food_allergen_map[allergen]])]
else:
    filtered_by_allergies = suitable_foods  # Ако няма алергии, не филтрираме

# 5. Филтриране по непоносимости
if user_intolerances != ['No']:  # Проверка дали има непоносимости
    filtered_by_intolerances = filtered_by_allergies[~filtered_by_allergies['name'].isin([item for intolerance in user_intolerances for item in food_intolerance_map[intolerance]])]
else:
    filtered_by_intolerances = filtered_by_allergies  # Ако няма непоносимости, не филтрираме

# 6. Филтриране по здравословно състояние
if user_health_condition == 'Hypertension':
    filtered_by_health = filtered_by_intolerances[
        (filtered_by_intolerances['energy_kcal'] < 0.554) & 
        (filtered_by_intolerances['sugars_g'] < 0.105)
    ]
elif user_health_condition == 'Heart Disease':
    filtered_by_health = filtered_by_intolerances[
        (filtered_by_intolerances['total_fat_g'] < 0.15) & 
        (filtered_by_intolerances['sugars_g'] < 0.052)
    ]
elif user_health_condition == 'Diabetes':
    filtered_by_health = filtered_by_intolerances[
        (filtered_by_intolerances['sugars_g'] < 0.052) & 
        (filtered_by_intolerances['carbohydrates_g'] < 0.303)
    ]
else:
    filtered_by_health = filtered_by_intolerances  # Ако няма здравословно състояние, не филтрираме допълнително

# 7. Резултатът е списък от храни, които са подходящи за потребителя
final_recommendations = filtered_by_health


# Проверка
print('Храни според diet_type', len(suitable_foods))
print("Храни след филтриране по алергии:", len(filtered_by_allergies))
print("Храни след филтриране по непоносимости:", len(filtered_by_intolerances))
print("Храни след филтриране по здравословно състояние:", len(filtered_by_health))

# 9. Извеждаме крайни препоръки
print("Food recommendations for the user:")
print(final_recommendations[['name', 'type', 'energy_kcal', 'protein_g', 'total_fat_g', 'carbohydrates_g', 'fiber_g', 'sugars_g', 'description']])

Храни след филтриране по хранителен режим: 3414
Храни след филтриране по алергии: 3414
Храни след филтриране по непоносимости: 3414
Храни след филтриране по здравословно състояние: 3414


In [26]:
# # 1. Създаване на речник за типове диети
# diet_food_mapping = {
#     'Vegetarian': ['fruit', 'vegetable', 'oils and solid fats', 'dairy', 'egg', 'added sugars', 'grains', 'beverages', 'plant-based drink', 'vegetarian'],
#     'Flexitarian': ['fruit', 'vegetable', 'oils and solid fats', 'dairy', 'egg', 'added sugars', 'meat', 'grains', 'beverages', 'shellfish', 'plant-based drink', 'fish', 'vegetarian'],
#     'Omnivorous': ['fruit', 'vegetable', 'oils and solid fats', 'dairy', 'egg', 'added sugars', 'meat', 'grains', 'beverages', 'shellfish', 'plant-based drink', 'fish', 'vegetarian'],
#     'Pescatarian': ['fruit', 'vegetable', 'oils and solid fats', 'dairy', 'egg', 'added sugars', 'grains', 'beverages', 'shellfish', 'plant-based drink', 'fish', 'vegetarian'],
#     'Vegan': ['fruit', 'vegetable', 'grains', 'beverages', 'plant-based drink']
# }

# # 2. Примерни входни данни от потребителя
# user_diet_type = 'Omnivorous'  # Примерен тип диета
# user_allergies = ['No']  # Примерни алергии
# user_intolerances = ['No']  # Примерни непоносимости
# user_health_condition = 'No'  # Примерно здравословно състояние

# # Списък с храни, които съдържат алергени
# food_allergen_map = {
#     'Peanuts': ['Peanut Butter', 'Peanuts'],
#     'Shellfish': ['Crab' 'Lobster' 'Shrimp' 'Spiny lobster', 'Clam', 'Octopus', 'Oyster', 'Scallop', 'Squid'],
#     'Dairy': ['Cheese', 'Cheese fd', 'Cheese sprd', 'Cream', 'Milk', 'Milk shakes', 'Yogurt', 'Cheese sub',
#               'Cheese sau', 'Sour cream', 'Ice creams', 'Frozen yogurts', 'Kraft macaroni & cheese dinner original flavor', 
#               'Macaroni and cheese', 'Creamy drsng', 'Milk dssrt', 'Cheese product', 'Cheese food'],
#     'Eggs': ['Egg', 'Egg mix', 'Eggnog-flavor mix', 'Egg custards', 'Scrambled eggs&sausage w/hashed brown potatoes', 'Egg rolls', 'Eggs'],
#     'Soybeans': ['Tofu', 'Soymilk', 'Soymilk (all flavors)', 'Soy sau made from soy&wheat (shoyu)'],
#     'Sesame': ['Tahini', 'Sesame seeds', 'Sesame'],
#     'Wheat': ['Cereals rte', 'Cereals', 'Cereals ready to eat', 'Cereal wafer straws', 'Bagel', 'Bagels', 'Tortillas', 'Tortilla chips',
#              'Wheat bran', 'Wheat germ', 'Wheat flour', 'Wheat flr', 'Macaroni', 'Wheat flours'],
#     'Fish': ['Salmon', 'Sea bass', 'Shark', 'Tuna', 'Whitefish', 'Salmonberries'],
#     # Добавете други алергени според нуждите
# }

# # Списък с храни, които съдържат непоносимости
# food_intolerance_map = {
#     'Lactose': ['Cheese', 'Cheese fd', 'Cheese sprd', 'Cream', 'Milk', 'Milk shakes', 'Yogurt', 'Cheese sub',
#               'Cheese sau', 'Sour cream', 'Ice creams', 'Frozen yogurts', 'Kraft macaroni & cheese dinner original flavor', 
#               'Macaroni and cheese', 'Creamy drsng', 'Milk dssrt', 'Cheese product', 'Cheese food'],
#     'Gluten': ['Cereals rte', 'Cereals', 'Cereals ready to eat', 'Cereal wafer straws', 'Bagel', 'Bagels', 'Tortillas', 'Tortilla chips',
#              'Wheat bran', 'Wheat germ', 'Wheat flour', 'Wheat flr', 'Macaroni', 'Wheat flours'],
#     'Fructose': ['Apple', 'Apricot', 'Avocado', 'Banana', 'Blackberries', 'Blueberry', 'Carambola', 'Cherimoya fruit', 'Cherry fruit', 
#                  'Clementine', 'Cranberries', 'Currants Black', 'Currants', 'Date nutrition: Dates', 'Dates', 'Durian', 'Fig nutrition: Figs',
#                  'Grapefruit', 'Grape nutrition: Grapes', 'Groundcherries', 'Guava', 'Gooseberries', 'Jackfruit', 'Kiwi nutriion: Kiwifruit',
#                  'Kumquat', 'Lemon nutrition', 'Lime', 'Litchis or Lychees', 'Mandarin', 'Mango', 'Melons', 'Mulberries', 'Nectarine', 'Oranges',
#                  'Papaya', 'Passion fruit', 'Peaches', 'Pear', 'Persimmon nutrition: Persimmons', 'Pineapple', 'Plum', 'Pomegranate', 'Quince fruit', 
#                  'Raspberry', 'Strawberry', 'Tangerine', 'Watermelon'],
# }

# # 3. Филтриране на храните според тип диета
# # Извеждаме подходящите храни според диетата на потребителя
# suitable_foods = food_macronutrients_normalized[food_macronutrients_normalized['type'].isin(diet_food_mapping[user_diet_type])]

# # 4. Филтриране по алергии
# if user_allergies != ['No']:  # Проверка дали има алергии
#     filtered_by_allergies = suitable_foods[~suitable_foods['name'].isin([item for allergen in user_allergies for item in food_allergen_map[allergen]])]
# else:
#     filtered_by_allergies = suitable_foods  # Ако няма алергии, не филтрираме

# # 5. Филтриране по непоносимости
# if user_intolerances != ['No']:  # Проверка дали има непоносимости
#     filtered_by_intolerances = filtered_by_allergies[~filtered_by_allergies['name'].isin([item for intolerance in user_intolerances for item in food_intolerance_map[intolerance]])]
# else:
#     filtered_by_intolerances = filtered_by_allergies  # Ако няма непоносимости, не филтрираме

# # 6. Филтриране по здравословно състояние
# if user_health_condition == 'Hypertension':
#     filtered_by_health = filtered_by_intolerances[
#         (filtered_by_intolerances['energy_kcal'] < 0.554) & 
#         (filtered_by_intolerances['sugars_g'] < 0.105)
#     ]
# elif user_health_condition == 'Heart Disease':
#     filtered_by_health = filtered_by_intolerances[
#         (filtered_by_intolerances['total_fat_g'] < 0.15) & 
#         (filtered_by_intolerances['sugars_g'] < 0.052)
#     ]
# elif user_health_condition == 'Diabetes':
#     filtered_by_health = filtered_by_intolerances[
#         (filtered_by_intolerances['sugars_g'] < 0.052) & 
#         (filtered_by_intolerances['carbohydrates_g'] < 0.303)
#     ]
# else:
#     filtered_by_health = filtered_by_intolerances  # Ако няма здравословно състояние, не филтрираме допълнително

# # 7. Резултатът е списък от храни, които са подходящи за потребителя
# final_recommendations = filtered_by_health

# # Проверка
# print('Храни според diet_type', len(suitable_foods))
# print("Храни след филтриране по алергии:", len(filtered_by_allergies))
# print("Храни след филтриране по непоносимости:", len(filtered_by_intolerances))
# print("Храни след филтриране по здравословно състояние:", len(filtered_by_health))

Обяснение:
Хипертония (Hypertension): При хора с хипертония се препоръчва да се ограничи калорийният прием, да се избягват храни с високо съдържание на захар и да се търсят храни с високо съдържание на фибри.
Сърдечни заболявания (Heart Disease): За хора със сърдечни заболявания трябва да се избягват храни с високо съдържание на мазнини и захар, като същевременно да се търсят храни с високо съдържание на фибри и ниско съдържание на мазнини.
Диабет (Diabetes): При диабет е важно да се ограничат въглехидратите и захарите, като същевременно да се приемат храни с повече фибри и протеини, които подпомагат контрола на кръвната захар.

## Train Test Split

In [27]:
# Разделяне на данните за класификацията на диети (dietry_habits_normalized)
X = dietry_habits_normalized.drop(['diet_type_encoded'], axis=1)  # или друга целева променлива, която използваш
y = dietry_habits_normalized['diet_type_encoded']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [28]:
print("Размер на тренировъчния сет:", X_train.shape)
print("Размер на тестовия сет:", X_test.shape)

Размер на тренировъчния сет: (4008, 19)
Размер на тестовия сет: (1002, 19)
